In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from scipy import signal

class Coefficients:
    taps = []

    def Generate(fs, fc, N):
        # Compute sinc filter
        nyquistFreq = fs / 2
        cutoffNormalized = fc / nyquistFreq
        h = np.sinc(cutoffNormalized * (np.arange(N) - (N - 1) / 2))

        # Apply window
        # h *= signal.windows.boxcar(int(N))
        # h *= signal.windows.blackman(int(N))
        h *= signal.windows.barthann(int(N))

        # Normalize to get unity gain
        h /= np.sum(h)

        # The CMSIS FIR requires the coefficients to be in time reversed order
        h = np.flip(h)

        return h
    
    def Evaluate(taps, fs):
        # Compute the frequency response of the filter
        nyquistFreq = fs / 2
        w, h = signal.freqz(taps)

        # Convert frequency to Hz
        freqHz = w * nyquistFreq / np.pi

        # Impulse response
        plt.figure(figsize=(10, 5))
        plt.plot(taps, '--')
        plt.grid()
        plt.title('Impulse response')
        plt.ylabel('Amplitude')
        plt.xlabel('Sample number')
        plt.show()        

        # Frequency response
        plt.figure(figsize=(10, 5))
        plt.plot(freqHz, abs(h))
        plt.grid()
        plt.title('Frequency response')
        plt.ylabel('Gain')
        plt.xlabel('Freq, Hz')
        plt.show()

        # Phase response
        plt.figure(figsize=(10, 5))
        anglesRad = (np.unwrap(np.angle(h)))
        plt.plot(freqHz, anglesRad, color='red')
        
        plt.grid()
        plt.title('Phase response')
        plt.ylabel('Angle, rad')
        plt.xlabel('Freq, Hz')
        plt.show()

# UI interraction
def GenerateBtn_ClickEvent(a):
    Coefficients.taps = Coefficients.Generate(sampleRateInput.value, cutoffFrequencyInput.value, filterLength.value)
    for sample in Coefficients.taps:
        print(f'{sample:.8f}', end=',\n')

def EvaluateBtn_ClickEvent(a):
    if Coefficients.taps.__len__() > 0:
        Coefficients.Evaluate(Coefficients.taps, sampleRateInput.value)

GenerateBtn = widgets.Button(center = True, description = "Generate")
GenerateBtn.on_click(GenerateBtn_ClickEvent)
EvaluateBtn = widgets.Button(center = True, description = "Evaluate")
EvaluateBtn.on_click(EvaluateBtn_ClickEvent)
BtnBox = HBox([GenerateBtn, EvaluateBtn])

sampleRateInput = widgets.FloatText(description = 'fs:', value = '48000')
cutoffFrequencyInput = widgets.FloatText(description = 'fc:', value = '6000')
filterLength = widgets.FloatText(description = 'length:', value = '255')
inputBox = VBox([sampleRateInput, cutoffFrequencyInput, filterLength, BtnBox])
display(inputBox)